## Setup

In [1]:
%load_ext sql

There's a new jupysql version available (0.10.11), you're running 0.10.10. To upgrade: pip install jupysql --upgrade
Deploy Panel apps for free on Ploomber Cloud! Learn more: https://ploomber.io/s/signup


In [ ]:
%config SqlMagic.autopandas = False

In [2]:
from dotenv import find_dotenv
from dotenv import load_dotenv
load_dotenv(find_dotenv())

True

In [3]:
import google.auth
credentials, project_id = google.auth.default()

In [4]:
import os
from google.cloud.sql.connector import Connector
import sqlalchemy

# initialize Connector object
connector = Connector()

# function to return the database connection object
def get_conn():
    conn = connector.connect(
        INSTANCE_CONNECTION_NAME,
        "pg8000",
        user=DB_USER,
        password=DB_PASSWORD,
        db=DB_NAME
    )
    return conn

def get_pool():
    # create connection pool with 'creator' argument to our connection object function
    pool = sqlalchemy.create_engine(
        "postgresql+pg8000://",
        creator=get_conn,
    )
    
    return pool

## Create Business Connection

In [5]:
DB_USER = os.getenv("BUSINESS_USERNAME", "")
DB_PASSWORD = os.getenv("BUSINESS_PASSWORD", "")
DB_NAME = os.getenv("BUSINESS_NAME", "")
INSTANCE_CONNECTION_NAME = os.getenv("BUSINESS_CONNECTION", "")

business_pool = get_pool()
%sql business_pool --alias business
%sql business

In [6]:
%%sql
select now();

Running query in 'business'

1 rows affected.

now
2024-07-10 15:53:04.031259+00:00


## Create Warehouse Connection

In [19]:
# DB_USER = os.getenv("WAREHOUSE_USERNAME", "")
# DB_PASSWORD = os.getenv("WAREHOUSE_PASSWORD", "")
# DB_NAME = os.getenv("WAREHOUSE_NAME", "")
# INSTANCE_CONNECTION_NAME = os.getenv("WAREHOUSE_CONNECTION", "")

# print(INSTANCE_CONNECTION_NAME)

# warehouse_pool = get_pool()
# %sql warehouse_pool --alias warehouse
# %sql warehouse

In [20]:
%%sql
select now();

Running query in 'business'

1 rows affected.

now
2024-07-05 16:46:57.471086+00:00


## List Database Connections

In [21]:
%sql --connections

current,url,alias
*,postgresql+pg8000://,business


## Query Business Database

In [84]:
%sql business
%config SqlMagic.displaylimit = None

displaylimit: Value None will be treated as 0 (no limit)

In [90]:
%%sql business_data <<
select * from business

Running query in 'business'

17 rows affected.

In [91]:
business_data

,id,legal_name,legal_type,identifier,tax_id,email,founding_date,ar_reminder_flag,last_ar_reminder_year
0,6,WESTWOOD HOUSING CO-OPERATIVE,CP,CP0001824,None,None,None,None,None
1,7,AAA,BC,BC0871227,None,None,None,None,None
2,8,AC BC 2022.DEC.6 18.24 TEST CORP.,BC,BC0871274,None,None,None,None,None
3,10,0870638 B.C. LTD.,BC,BC0870638,None,None,None,None,None
4,11,0870574 B.C. LTD.,BC,BC0870574,None,None,None,None,None
5,12,WAFFLES R U LTD.,BC,BC0870626,None,None,None,None,None
6,13,0870596 B.C. LTD.,BC,BC0870596,None,None,None,None,None
7,14,FRASER VALLEY CONTRACTING INCORPORATED LIMITED,BC,BC0870630,None,None,None,None,None
8,15,PUPPY PATROL INVESTMENTS INC.,C,C0842268,None,None,None,None,None
9,16,WONEWOMANWORKS LTD.,C,C0842323,None,None,None,None,None


# Rollback the New Changes

In [92]:
identifier = "BC0870574"
business_id = "11"

In [93]:
%config SqlMagic.autopandas = True

In [108]:
%%sql filings <<
select * from filing WHERE business_id={{business_id}}

Running query in 'business'

1 rows affected.

In [109]:
filingId = filings.iloc[-1].id
filingId

np.int64(97)

In [110]:
%%sql colin_event_ids <<
SELECT * FROM colin_event_ids WHERE filing_id={{filingId}}

Running query in 'business'

1 rows affected.

In [111]:
colin_event_ids

,colin_event_id,filing_id
0,9180257,97


In [112]:
unique_event_ids = []
for index, row in colin_event_ids.iterrows():
    if row["colin_event_id"] not in unique_event_ids:
        unique_event_ids.append(str(row["colin_event_id"].item()))

unique_event_ids

[np.int64(9180257)]

In [113]:
%%sql
DELETE FROM colin_event_ids WHERE filing_id = {{filingId}}

Running query in 'business'

1 rows affected.

""


In [114]:
%%sql deleted <<
DELETE FROM filing WHERE id={{filingId}}

Running query in 'business'

1 rows affected.

# Rollback Colin

In [8]:
import requests
from datetime import datetime

COLIN_API_URL = os.getenv("COLIN_API_URL", "")
COLIN_CLIENT = os.getenv("COLIN_CLIENT", "")
COLIN_SECRET = os.getenv("COLIN_SECRET", "")

colin_api_endpoint = f"{COLIN_API_URL}/reset/by_event_id"

#Get Token
token_url = os.getenv("AUTH_SVC_URL", "")
timeout = int(os.getenv("AUTH_SVC_TIMEOUT", 20))

data = "grant_type=client_credentials"

# get service account token
res = requests.post(
  url=token_url,
  data=data,
  headers={"content-type": "application/x-www-form-urlencoded"},
  auth=(COLIN_CLIENT, COLIN_SECRET),
  timeout=timeout,
)

token = res.json().get("access_token")

reset_details = requests.post(
    colin_api_endpoint,
    headers={"content-type": "application/json", "Authorization": "Bearer "+token},
    json={"event_ids": unique_event_ids},
).json()

reset_details

{'message': 'Reset for event ids ["9180257"]'}

## SBCAUTH Keycloak

This is going to have to be manual since sbcauth only does soft delete and keycloak only supports modification via an admin api key. (Previously this had to be admin on master which won't happen TBD if thats still the case)


Manual - Delete the user in keycloak.

### Find the user id using last name (example based on integration test user)
```sql
select id into user_id_value from users where last_name = 'WU';
```

### Delete from SBC Auth DB
Note replace 2223 below with the user id
```sql
begin;
update entities set modified_by_id = null where modified_by_id = 2223;
update memberships set created_by_id = null where created_by_id = 2223;
delete from product_subscriptions where created_by_id = 2223;
delete from memberships where user_id = 2223;
delete from contact_links where created_by_id = 2223 or user_id = 2223;
delete from affiliations where created_by_id = 2223;
delete from orgs where created_by_id = 2223;
delete from contact_links where user_id = 2223;
delete from contact_links where created_by_id = 2223;
delete from contacts where created_by_id = 2223;
delete from users where id = 2223;
```

